In [1]:
## Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
## Reading csv and excel files into dataframes
tm145 = pd.read_csv('../data/race_data/TelemetryData_14577742557187250604.csv', low_memory=False)
sd145 = pd.read_csv('../data/race_data/SessionData_14577742557187250604.csv', low_memory=False)
sd102 = pd.read_csv('../data/race_data/SessionData_10230136787177318441.csv', low_memory=False)
sd108 = pd.read_csv('../data/race_data/SessionData_1087987310493855854.csv', low_memory=False)
sd117 = pd.read_csv('../data/race_data/SessionData_11752605864096667831.csv', low_memory=False)
pilot1_102 = pd.read_excel('../data/excel_files/driver_1_tm102.xlsx')
pilot1_108 = pd.read_excel('../data/excel_files/driver_1_tm108.xlsx')
pilot1_117 = pd.read_excel('../data/excel_files/driver_1_tm117.xlsx')

##### I pushed all .csv files from the dataset onto a postgres database for manipulation using src/csv_to_db.py
##### Datasets filtered by pilot were created using sql_queries/* and src/convert_sqls.py

In [9]:
tm145.columns

Index(['sessionTime', 'frameIdentifier', 'pilot_index', 'worldPositionX',
       'worldPositionY', 'worldPositionZ', 'worldVelocityX', 'worldVelocityY',
       'worldVelocityZ', 'worldForwardDirX', 'worldForwardDirY',
       'worldForwardDirZ', 'worldRightDirX', 'worldRightDirY',
       'worldRightDirZ', 'gForceLateral', 'gForceLongitudinal',
       'gForceVertical', 'yaw', 'pitch', 'roll', 'speed', 'throttle', 'steer',
       'brake', 'clutch', 'gear', 'engineRPM', 'drs', 'brakesTemperature',
       'tyresSurfaceTemperature', 'tyresInnerTemperature', 'engineTemperature',
       'tyresPressure', 'surfaceType', 'fuelMix', 'pitLimiterStatus',
       'fuelInTank', 'fuelRemainingLaps', 'tyresWear', 'actualTyreCompound',
       'tyresDamage', 'ersStoreEnergy', 'ersDeployMode',
       'ersHarvestedThisLapMGUK', 'ersHarvestedThisLapMGUH',
       'ersDeployedThisLap', 'carPosition', 'currentLapTime', 'currentLapNum',
       'lapDistance', 'totalDistance', 'pitStatus', 'sector', 'driverStatus',